In [1]:
import os
import sys
import random
import mne
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE

import pandas as pd
import logging
import time
import json

from datetime import datetime, timezone, timedelta

from collections import Counter

In [2]:
# f = mne.io.read_raw_edf("./Yingxf/Yingxf.edf")
# f = mne.io.read_raw_edf("./Liucx/Liucx.edf")

# data_path = "./SJTU/Chenc/Chenc.edf"
# f = mne.io.read_raw_edf(data_path)
# seiz_start_time = [
#     datetime(2023, 9, 13, 17, 28, 30, tzinfo=timezone.utc),
#     # datetime(2023, 9, 13, 17, 42, 29, tzinfo=timezone.utc),
#     # datetime(2023, 9, 13, 17, 55, 42, tzinfo=timezone.utc),
    
#     datetime(2023, 9, 13, 21, 52, 1, tzinfo=timezone.utc),
#     # datetime(2023, 9, 13, 21, 57, 30, tzinfo=timezone.utc),

#     datetime(2023, 9, 13, 22, 57, 20, tzinfo=timezone.utc),
#     # datetime(2023, 9, 13, 23, 05, 18, tzinfo=timezone.utc),
#     # datetime(2023, 9, 13, 23, 17, 04, tzinfo=timezone.utc),

#     datetime(2023, 9, 14, 1, 7, 52, tzinfo=timezone.utc),
#     # datetime(2023, 9, 14, 1, 10, 42, tzinfo=timezone.utc),
#     # datetime(2023, 9, 14, 1, 13, 10, tzinfo=timezone.utc),
#     # datetime(2023, 9, 14, 1, 15, 45, tzinfo=timezone.utc)
# ]
# seiz_end_time = [
#     # datetime(2023, 9, 13, 17, 28, 32, tzinfo=timezone.utc),
#     # datetime(2023, 9, 13, 17, 42, 31, tzinfo=timezone.utc),
#     datetime(2023, 9, 13, 17, 55, 43, tzinfo=timezone.utc),
    
#     # datetime(2023, 9, 13, 21, 52, 03, tzinfo=timezone.utc),
#     datetime(2023, 9, 13, 21, 57, 31, tzinfo=timezone.utc),

#     # datetime(2023, 9, 13, 22, 57, 22, tzinfo=timezone.utc), 
#     # datetime(2023, 9, 13, 23, 05, 20, tzinfo=timezone.utc),
#     datetime(2023, 9, 13, 23, 17, 5, tzinfo=timezone.utc),

#     # datetime(2023, 9, 14, 1, 7, 53, tzinfo=timezone.utc),
#     # datetime(2023, 9, 14, 1, 10, 43, tzinfo=timezone.utc),
#     # datetime(2023, 9, 14, 1, 13, 11, tzinfo=timezone.utc),
#     datetime(2023, 9, 14, 1, 15, 48, tzinfo=timezone.utc)
# ]




data_path = "./SJTU/Dingdw/Dingdw.edf"
f = mne.io.read_raw_edf(data_path)
seiz_start_time = [
    datetime(2023, 5, 19, 21, 7, 16, tzinfo=timezone.utc),
    datetime(2023, 5, 19, 22, 9, 58, tzinfo=timezone.utc),
    datetime(2023, 5, 19, 23, 47, 8, tzinfo=timezone.utc),
    datetime(2023, 5, 20, 0, 25, 59, tzinfo=timezone.utc),
    datetime(2023, 5, 20, 1, 2, 11, tzinfo=timezone.utc),
    datetime(2023, 5, 20, 1, 43, 13, tzinfo=timezone.utc),
    datetime(2023, 5, 20, 4, 48, 44, tzinfo=timezone.utc),
    datetime(2023, 5, 20, 6, 26, 6, tzinfo=timezone.utc)
]

seiz_end_time = [
    datetime(2023, 5, 19, 21, 9, 0, tzinfo=timezone.utc),
    datetime(2023, 5, 19, 22, 11, 18, tzinfo=timezone.utc),
    datetime(2023, 5, 19, 23, 49, 57, tzinfo=timezone.utc),
    datetime(2023, 5, 20, 0, 27, 19, tzinfo=timezone.utc),
    datetime(2023, 5, 20, 1, 4, 19, tzinfo=timezone.utc),
    datetime(2023, 5, 20, 1, 53, 56, tzinfo=timezone.utc),
    datetime(2023, 5, 20, 4, 54, 39, tzinfo=timezone.utc),
    datetime(2023, 5, 20, 6, 28, 4, tzinfo=timezone.utc)
]




# f = mne.io.read_raw_edf("./Chenzh/Chenzh.edf")
# f = mne.io.read_raw_edf("./Dingdw/Dingdw.edf")
# f = mne.io.read_raw_edf("./Dingdw/Suny.edf")
# f = mne.io.read_raw_edf("./Guljy/Guljy.edf")
# f = mne.io.read_raw_edf("./Xuyx/Xuyx.edf")
# f = mne.io.read_raw_edf("./Huangfs/Huangfs.edf")
# f = mne.io.read_raw_edf("./Majp/Majp.edf")

Extracting EDF parameters from E:\EEG\SJTU\Dingdw\Dingdw.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\95842\AppData\Local\Temp\ipykernel_1948\3360425519.py:45: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  f = mne.io.read_raw_edf(data_path)
C:\Users\95842\AppData\Local\Temp\ipykernel_1948\3360425519.py:45: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  f = mne.io.read_raw_edf(data_path)


In [3]:
f.info['meas_date'], len(seiz_start_time), len(seiz_end_time)

(datetime.datetime(2023, 5, 19, 13, 51, tzinfo=datetime.timezone.utc), 8, 8)

In [4]:
for i in range( len(seiz_start_time) ):
    print( seiz_end_time[i] - seiz_start_time[i], seiz_start_time[i], seiz_end_time[i] )

0:01:44 2023-05-19 21:07:16+00:00 2023-05-19 21:09:00+00:00
0:01:20 2023-05-19 22:09:58+00:00 2023-05-19 22:11:18+00:00
0:02:49 2023-05-19 23:47:08+00:00 2023-05-19 23:49:57+00:00
0:01:20 2023-05-20 00:25:59+00:00 2023-05-20 00:27:19+00:00
0:02:08 2023-05-20 01:02:11+00:00 2023-05-20 01:04:19+00:00
0:10:43 2023-05-20 01:43:13+00:00 2023-05-20 01:53:56+00:00
0:05:55 2023-05-20 04:48:44+00:00 2023-05-20 04:54:39+00:00
0:01:58 2023-05-20 06:26:06+00:00 2023-05-20 06:28:04+00:00


In [5]:

print('seiz_start_time = [')
for i in range( len(seiz_start_time) ):
    temp_str = f'    datetime({seiz_start_time[i].year}, {seiz_start_time[i].month}, {seiz_start_time[i].day}, {seiz_start_time[i].hour}, {seiz_start_time[i].minute}, {seiz_start_time[i].second}, tzinfo=timezone.utc)'
    if (i != 0) and (seiz_start_time[i]-seiz_end_time[i-1] > timedelta(minutes=30) ):
        print()
    if i != len(seiz_start_time)-1:
        print( temp_str+',' )
    else:
        print( temp_str)
print(']')

seiz_start_time = [
    datetime(2023, 5, 19, 21, 7, 16, tzinfo=timezone.utc),

    datetime(2023, 5, 19, 22, 9, 58, tzinfo=timezone.utc),

    datetime(2023, 5, 19, 23, 47, 8, tzinfo=timezone.utc),

    datetime(2023, 5, 20, 0, 25, 59, tzinfo=timezone.utc),

    datetime(2023, 5, 20, 1, 2, 11, tzinfo=timezone.utc),

    datetime(2023, 5, 20, 1, 43, 13, tzinfo=timezone.utc),

    datetime(2023, 5, 20, 4, 48, 44, tzinfo=timezone.utc),

    datetime(2023, 5, 20, 6, 26, 6, tzinfo=timezone.utc)
]


In [6]:
print('seiz_end_time = [')
for i in range( len(seiz_start_time) ):
    temp_str = f'    datetime({seiz_end_time[i].year}, {seiz_end_time[i].month}, {seiz_end_time[i].day}, {seiz_end_time[i].hour}, {seiz_end_time[i].minute}, {seiz_end_time[i].second}, tzinfo=timezone.utc)'
    if (i != 0) and (seiz_start_time[i]-seiz_end_time[i-1] > timedelta(minutes=30) ):
        print()
    if i != len(seiz_start_time)-1:
        print( temp_str+',' )
    else:
        print( temp_str)
print(']')

seiz_end_time = [
    datetime(2023, 5, 19, 21, 9, 0, tzinfo=timezone.utc),

    datetime(2023, 5, 19, 22, 11, 18, tzinfo=timezone.utc),

    datetime(2023, 5, 19, 23, 49, 57, tzinfo=timezone.utc),

    datetime(2023, 5, 20, 0, 27, 19, tzinfo=timezone.utc),

    datetime(2023, 5, 20, 1, 4, 19, tzinfo=timezone.utc),

    datetime(2023, 5, 20, 1, 53, 56, tzinfo=timezone.utc),

    datetime(2023, 5, 20, 4, 54, 39, tzinfo=timezone.utc),

    datetime(2023, 5, 20, 6, 28, 4, tzinfo=timezone.utc)
]


In [7]:
def calculate_accuracy_np(y_true, y_pred):
    labels_true = y_true#np.argmax(y_true, axis=1)  # 将真实标签转换为标签
    labels_pred = y_pred#np.argmax(y_pred, axis=1)  # 将预测结果转换为标签
    accuracy = np.sum(labels_true == labels_pred) / len(labels_true)  # 计算准确率
    return accuracy

def calculate_sensitivity_np(y_true, y_pred, class_index):
    labels_true = y_true#np.argmax(y_true, axis=1)  # 将真实标签转换为标签
    labels_pred = y_pred#np.argmax(y_pred, axis=1)  # 将预测结果转换为标签
    positive_labels = (labels_true == class_index)  # 将指定类别的标签设为1，其他类别设为0
    true_positive = np.sum((labels_pred == labels_true) * positive_labels)  # 统计真阳性的数量
    false_negative = np.sum((labels_pred != labels_true) * positive_labels)  # 统计假阴性的数量
    if (true_positive + false_negative) == 0:
        return -1
    sensitivity = true_positive / (true_positive + false_negative)  # 计算灵敏度
    return sensitivity

def get_names( ch_num, l ):
    name_list = []
    for i in range(ch_num):
        name_list.append( l[i]['name'] )
        # print(l[i]['name'][0])
    return name_list


def top_n_elements(lists, n=6):
    # 创建一个 Counter 对象以统计所有子列表中的元素
    overall_counter = Counter()
    
    # 遍历每个子列表并更新计数
    for sublist in lists:
        overall_counter.update(get_names(n, sublist))

    # 获取前 n 名元素
    top_elements_with_counts = overall_counter.most_common(n)

    top_elements = [element for element, count in top_elements_with_counts]
    counts = [count for element, count in top_elements_with_counts]
    
    return top_elements, counts

class ListMerger:
    def __init__(self, list1, list2):
        self.list1 = list1
        self.list2 = list2

    def __getitem__(self, index):
        if isinstance(index, slice):
            # 处理切片
            return [self[i] for i in range(*index.indices(len(self)))]
        elif index < len(self.list1):
            return self.list1[index]
        else:
            return self.list2[index - len(self.list1)]

    def __len__(self):
        # 返回合并后的列表的总长度
        return len(self.list1) + len(self.list2)

    def __iter__(self):
        return iter(self.list1 + self.list2)

class MergedDataset(Dataset):
    def __init__(self, merger, labels, indices):
        """
        :param merger: ListMerger 实例，包含合并后的数据
        :param indices: 要使用的索引列表
        """
        self.merger = merger
        self.labels = labels
        self.indices = indices

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        # 获取索引
        index = self.indices[idx]
        # 返回合并后的特征和标签
        return self.merger[index], self.labels[index]

def do_overlap( original_seq, seg_len, overlap_len ):
    '''
    Input:
        original_seq: 2d tensor, [ch, seq_len]
        overlap_len: float, overlapping length of 2 successive segments
    Outputs:
        res: a list of tensors
    '''
    res = []

    for i in range( 0, original_seq.shape[1], seg_len-overlap_len ):
        if original_seq.shape[1] - i < seg_len:
            break
        res.append( original_seq[:, i:(i+seg_len)] )

    return res

def read_SJTU_data( raw, seiz_start_time, seiz_end_time, COMMON_CH, SEG_TIME=5, OVERLAP=True ):
    # f = mne.io.read_raw_edf(data_folder)
    f = raw.copy()
    f.pick( COMMON_CH )
    data = f.get_data()
    data = data * 1e4
    
    meas_start_time = f.info['meas_date']#.strftime('%Y-%m-%d %H:%M:%S')
    sfreq = f.info['sfreq']
    all_channels = f.info['ch_names'][:-1] # drop last channels 'MK'
    
    SPH = int(15 * 60 * sfreq)
    SEG_LEN = int( SEG_TIME * sfreq )

    segments_pre = []
    segments_inter = []

    total_len_pre = 0
    total_len_inter = 0

    last_end = -1
    seiz_start_bias_time = []
    seiz_end_bias_time = []
    for i in range( len(seiz_start_time) ):
        seiz_start_bias_time.append( int((seiz_start_time[i] - meas_start_time).total_seconds() * sfreq) )
        seiz_end_bias_time.append( int((seiz_end_time[i] - meas_start_time).total_seconds() * sfreq) )

        beg = seiz_start_bias_time[i] - SPH
        end = seiz_start_bias_time[i]
        # print(beg, end)
        if beg < 0:
            segments_pre.append( torch.tensor(data[:, 0:seiz_start_bias_time[i]], dtype=torch.float32) )
            total_len_pre += seizure_start[i]
        else:
            segments_pre.append( torch.tensor(data[:, beg:end+1], dtype=torch.float32) )
            total_len_pre += segments_pre[-1].shape[1]
            segments_inter.append( torch.tensor(data[:, (last_end+1) : beg], dtype=torch.float32) )
            total_len_inter += segments_inter[-1].shape[1]
        last_end = seiz_end_bias_time[i]
        
    segments_inter.append( torch.tensor(data[:, (last_end+1):], dtype=torch.float32) ) 
    total_len_inter += segments_inter[-1].shape[1]
    # seiz_start_bias_time, seiz_end_bias_time
    
    segments_pre_overlap = []
    segments_inter_no_overlap = []
    step_for_overlap = int( SEG_LEN / ( total_len_inter / total_len_pre ) )
    # print("step_for_overlap:", step_for_overlap)
    # print(total_len_inter, total_len_pre)
    while segments_inter or segments_pre:
        if segments_pre:
            # segments_pre_overlap.append( do_overlap( seg_pre, SEG_LEN, OVERLAP_TIME*NEW_SAMP_RATE ) )
            if OVERLAP:
                segments_pre_overlap.extend( do_overlap( segments_pre[0], SEG_LEN, SEG_LEN-step_for_overlap) ) #OVERLAP_TIME*NEW_SAMP_RATE ) )
            else:
                segments_pre_overlap.extend( do_overlap( segments_pre[0], SEG_LEN, 0) )
            segments_pre.pop(0)
            # print( len(segments_pre_overlap) )
        if segments_inter:
            # segments_inter_no_overlap.append( do_overlap( segments_inter[0], SEG_LEN, OVERLAP_TIME*NEW_SAMP_RATE ) )
            segments_inter_no_overlap.extend( do_overlap( segments_inter[0], SEG_LEN, 0 ) )
            segments_inter.pop(0)
            # print( len(segments_inter_no_overlap) )
    
    
    return segments_inter_no_overlap, segments_pre_overlap



In [8]:
TOTAL_ITERATION = 30
SEG_TIME = 5

total_acc_list = []
for iteration in range(TOTAL_ITERATION):
    start_time = time.time()
    
    acc_list = []
    for COMMON_CH in f.info['ch_names'][:-1]:
        segments_inter, segments_pre = read_SJTU_data( f, seiz_start_time, seiz_end_time, COMMON_CH, SEG_TIME, OVERLAP=False )
        # print( segments_inter[0].shape, segments_pre[0].shape )
        segments = segments_inter + segments_pre #ListMerger( segments_inter, segments_pre )
        seg_labels = [0] * len(segments_inter) + [1] * len(segments_pre)
        one_hot_enc = LabelBinarizer()
        seg_labels = one_hot_enc.fit_transform(seg_labels) #torch.nn.functional.one_hot(torch.tensor(seg_labels), num_classes=2)

        segments = np.array(segments)
        seg_labels = np.array(seg_labels)
    
        segments_for_train,  segments_for_test, seg_labels_for_train, seg_labels_for_test = train_test_split(segments, seg_labels, test_size=0.3, shuffle=True)
        
        seg_train_pca_list = []
        seg_test_pca_list = []
        pca = PCA(n_components=60)
        segments_for_train_pca = pca.fit_transform(segments_for_train.squeeze())
        segments_for_test_pca = pca.transform(segments_for_test.squeeze())
        
        smote = SMOTE()#(random_state=42)
        segments_for_train_pca, seg_labels_for_train = smote.fit_resample(segments_for_train_pca, seg_labels_for_train)
    
        clf = DecisionTreeClassifier()#random_state=0)
        clf.fit(segments_for_train_pca, seg_labels_for_train)
    
        y_pred = clf.predict(segments_for_test_pca)
    
        sens_i = calculate_sensitivity_np(seg_labels_for_test.squeeze(), y_pred, 1)
        accuracy = calculate_accuracy_np(seg_labels_for_test.squeeze(), y_pred)
    
        temp = {}
        temp['sens'] = sens_i
        temp['acc'] = accuracy
        temp['name'] = COMMON_CH
        acc_list.append(temp)

    sorted_acc_list = sorted( acc_list, key=lambda x: x["acc"], reverse=True)
    total_acc_list.append(sorted_acc_list)
    
    end_time = time.time()
    print(iteration, end_time-start_time, 's')
    
with open('SJTU/' + os.path.basename(data_path).split('.')[0]+'_sel_ch_30iter_with_SMOTE.json', 'w') as json_file:
    json.dump(total_acc_list, json_file)       

0 60.232354402542114 s
1 59.722538471221924 s
2 59.042083501815796 s
3 59.974239110946655 s
4 59.43891429901123 s
5 59.890865087509155 s
6 59.43694090843201 s
7 59.366936683654785 s
8 58.72571420669556 s
9 59.22095012664795 s
10 59.080167293548584 s
11 58.766242027282715 s
12 58.13712239265442 s
13 59.510873794555664 s
14 58.87479496002197 s
15 57.63569521903992 s
16 59.93304228782654 s
17 59.219544649124146 s
18 59.7323784828186 s
19 59.92482662200928 s
20 59.02675700187683 s
21 59.03997325897217 s
22 58.77252554893494 s
23 60.38619661331177 s
24 60.83774185180664 s
25 59.21881699562073 s
26 60.21929144859314 s
27 58.91010093688965 s
28 60.32468509674072 s
29 59.439868211746216 s


In [9]:
'SJTU/'+os.path.basename(data_path).split('.')[0]+'_sel_ch_30iter_with_SMOTE.json'

'SJTU/Dingdw_sel_ch_30iter_with_SMOTE.json'

In [10]:
with open('SJTU/'+os.path.basename(data_path).split('.')[0]+'_sel_ch_30iter_with_SMOTE.json', 'r') as json_file:
    loaded_data = json.load(json_file)
top_n_elements( loaded_data, 8 )

(['EEG O1',
  'EEG T6',
  'EEG T5',
  'EEG F8',
  'EEG P4',
  'EEG C4',
  'EEG A2',
  'EEG A1'],
 [30, 30, 30, 29, 24, 20, 18, 16])

In [11]:
loaded_data

[[{'sens': 0.5653153153153153, 'acc': 0.847048496818082, 'name': 'EEG O1'},
  {'sens': 0.41784037558685444, 'acc': 0.8334430546412114, 'name': 'EEG T6'},
  {'sens': 0.42437923250564336, 'acc': 0.8290542023260917, 'name': 'EEG F8'},
  {'sens': 0.4779582366589327, 'acc': 0.8275181040157998, 'name': 'EEG T5'},
  {'sens': 0.463302752293578, 'acc': 0.8209348255431205, 'name': 'EEG A1'},
  {'sens': 0.34965034965034963, 'acc': 0.8200570550800965, 'name': 'EEG Fp2'},
  {'sens': 0.4375, 'acc': 0.8167654158437568, 'name': 'EEG Fp1'},
  {'sens': 0.4497716894977169, 'acc': 0.8158876453807329, 'name': 'EEG C4'},
  {'sens': 0.4433497536945813, 'acc': 0.8125960061443932, 'name': 'EEG P4'},
  {'sens': 0.392018779342723, 'acc': 0.8093043669080535, 'name': 'EEG A2'},
  {'sens': 0.39803439803439805, 'acc': 0.8055738424402019, 'name': 'EEG F3'},
  {'sens': 0.41067285382830626, 'acc': 0.804915514592934, 'name': 'EEG C3'},
  {'sens': 0.41743119266055045, 'acc': 0.8033794162826421, 'name': 'EEG F4'},
  {'sen